In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import os
import logging

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.4.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Thu_Jun_11_22:26:48_Pacific_Daylight_Time_2020
Cuda compilation tools, release 11.0, V11.0.194
Build cuda_11.0_bu.relgpu_drvr445TC445_37.28540450_0


In [5]:
weather = pd.read_csv("data/진주기상정보.csv", encoding = "cp949")
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [6]:
weather.rename(columns = {"일시" : "일자"}, inplace = True)
train.describe()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,2807.815768,157.913693,241.142739,274.117012,43.506224,890.334440,461.772614
std,171.264404,144.190572,43.532298,246.239651,109.937400,209.505057,139.179202
min,2601.000000,23.000000,41.000000,0.000000,0.000000,296.000000,0.000000
25%,2645.000000,71.000000,217.000000,4.000000,0.000000,758.000000,406.000000
50%,2760.000000,105.000000,245.000000,299.000000,0.000000,879.000000,483.000000
75%,2962.000000,185.000000,272.000000,452.000000,0.000000,1032.000000,545.000000
max,3305.000000,1224.000000,378.000000,1044.000000,533.000000,1459.000000,905.000000


In [7]:
# train_merge = pd.merge(train, weather, on = "일자", how = "left")
# test_merge = pd.merge(test, weather, on = "일자", how = "left")
# train_merge.describe()

In [8]:
train_merge = train
test_merge = test

In [9]:
!pip install lxml
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def getHoliday(year: int) -> pd.DataFrame:
    url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo"
    api_key_utf8 = "I2Mv6nH4CWf3VYYi83oy0EJO2upQgmp2GDI%2Fdbxh9ekXkRD4EjMSq0gWZxTy%2FNtAxnjV7soTkOW2xs1qYMdjBg%3D%3D"
    api_key_decode = parse.unquote(api_key_utf8)

    params = {
        "ServiceKey": api_key_decode,
        "solYear": year,
        "numOfRows": 100
    }

    response = requests.get(url, params=params)
    xml = BeautifulSoup(response.text, "html.parser")
    items = xml.find('items')
    item_list = []
    for item in items:
        item_dict = {
            "공휴일": 1, # 무슨 공휴일인지는 없어도 될 듯 하다
            "일자": datetime.strptime(item.find("locdate").text.strip(), '%Y%m%d')
        }
        item_list.append(item_dict)

    return pd.DataFrame(item_list)

In [10]:
holidays = pd.concat([getHoliday(year) for year in range(2016, 2022)])
holidays.describe()

,공휴일
count,103.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [11]:
from datetime import datetime, timedelta

def one_day_before(s):
    return str(datetime.strptime(str(s)[:10], "%Y-%m-%d").date() + timedelta(days = -1))

def one_day_after(s):
    return str(datetime.strptime(str(s)[:10], "%Y-%m-%d").date() + timedelta(days = 1))

holidays["공휴일전날"] = holidays["일자"].apply(one_day_before)
holidays["공휴일다음날"] = holidays["일자"].apply(one_day_after)

In [12]:
new_holidays = pd.concat([holidays["공휴일전날"], holidays["공휴일다음날"]])

new_holidays = pd.DataFrame(new_holidays, columns = ["일자"])

new_holidays["휴일전후날여부"] = 1
new_holidays.head(110)

,일자,휴일전후날여부
0,2015-12-31,1
1,2016-02-06,1
2,2016-02-07,1
3,2016-02-08,1
4,2016-02-09,1
...,...,...
2,2016-02-09,1
3,2016-02-10,1
4,2016-02-11,1
5,2016-03-02,1


In [13]:
train_merge = pd.merge(train_merge, new_holidays, how = 'left', on  = '일자')
test_merge = pd.merge(test_merge, new_holidays, how = 'left', on = '일자')

train_merge["휴일전후날여부"] = train_merge["휴일전후날여부"].fillna(0)
test_merge["휴일전후날여부"] = test_merge["휴일전후날여부"].fillna(0)

# keep submission source
submission = test_merge[["일자"]]

train_merge.drop(["일자"], inplace = True, axis = 1)
test_merge.drop(["일자"], inplace = True, axis = 1)
train_merge.head(50)

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전후날여부
0,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0.0
1,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0.0
2,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0.0
3,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,0.0
4,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,0.0
5,목,2601,383,143,417,0.0,팬케익/찐빵 우유/두유/주스 계란후라이 견과류죽/쌀밥 (쌀:국내산) 감자찌개 ...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 시래기국 훈제오리구이 도토리묵무침 쌈무/양...","참치회덮밥 (쌀,현미흑미:국내산) 맑은국 군만두 과일샐러드 락교 포기김치 (...",1045.0,550.0,1.0
6,금,2601,389,156,93,0.0,모닝롤/야채샌드 우유/두유/주스 계란후라이 고구마죽/쌀밥 (쌀:국내산) 봄동된...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 꽃게탕 돈육굴소스볶음 옥수수전 유채나물 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치콩나물국 미니함박 어묵볶음 물파래무침 ...",909.0,598.0,0.0
7,월,2601,87,204,482,0.0,모닝롤/치즈프레즐 우유/두유/주스 계란후라이 잣죽/쌀밥 (쌀:국내산) 민물새우...,쌀밥/잡곡밥 (쌀:국내산) 시금치국 닭감자조림 (닭고기:국내산) 연두부*양념장 ...,"쌀밥/잡곡밥 (쌀:국내산) 홍합미역국 등갈비김치찜 (돼지고기,김치:국내산) 임연수...",1268.0,672.0,0.0
8,화,2601,72,236,526,0.0,모닝롤/마늘빵 우유/두유/주스 계란후라이 단호박죽/쌀밥 (쌀:국내산) 어묵국 ...,쌀밥/잡곡밥 (쌀:국내산) 쇠고기무국 (쇠고기:호주산) 탕수어 (동태:러시아산) 오...,쌀밥/잡곡밥 (쌀:국내산) 된장찌개 쇠불고기 (쇠고기:호주산) 해파리겨자채 봄동...,1014.0,523.0,0.0
9,수,2601,78,250,23,0.0,모닝롤/참치샌드 우유/두유/주스 계란후라이 흑임자죽/쌀밥 (쌀:국내산) 북어계...,쌀밥/잡곡밥 (쌀:국내산) 냉이된장국 쇠고기장조림 (쇠고기:호주산) 통도라지구이 ...,볶음밥*자장소스 (쌀:국내산) 맑은국 새우또띠아 쨔샤이무침 요플레 포기김치 ...,916.0,588.0,0.0


In [14]:
train_merge = pd.get_dummies(train_merge, columns = ["휴일전후날여부"], prefix = '휴일전후날여부')
test_merge = pd.get_dummies(test_merge, columns = ["휴일전후날여부"], prefix = '휴일전후날여부')

In [15]:
train_merge["본사실근무자수"] = train_merge["본사정원수"] - train_merge["본사휴가자수"] - train_merge["본사출장자수"]/2 - train_merge["현본사소속재택근무자수"]
test_merge["본사실근무자수"] = test_merge["본사정원수"] - test_merge["본사휴가자수"] - test_merge["본사출장자수"]/2 - test_merge["현본사소속재택근무자수"]

del_features = ["본사정원수", "본사휴가자수", "본사출장자수", "현본사소속재택근무자수"]
train_merge.drop(del_features, inplace = True, axis = 1)
test_merge.drop(del_features, inplace = True, axis = 1)

In [16]:
train_merge = pd.get_dummies(train_merge, columns = ["요일"], prefix = '요일')
test_merge = pd.get_dummies(test_merge, columns = ["요일"], prefix = '요일')
train_merge.head()

,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,요일_화
0,238,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,1,0,2476.0,0,0,0,1,0
1,319,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,1,0,2464.5,0,0,0,0,1
2,111,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,1,0,2455.0,0,0,1,0,0
3,355,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,1,0,2387.0,0,1,0,0,0
4,34,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,1,0,2232.5,1,0,0,0,0


In [17]:
import re
def reg_modi(s):
    s = re.sub(pattern = r'\[[\S]*\]', repl = " ", string = s)
    s = re.sub(pattern = r'\([\S]*\)', repl = " ", string = s)
    s = re.sub(pattern = r'\s\s+', repl = " ", string = s)
    s = re.sub(pattern = r'\*', repl = " ", string = s)
    s = re.sub(pattern = r' $', repl = "", string = s)
    if len(s) < 15 or "개발의" in s or "가정의" in s: # issue
        return "X"
    return s

for menu in ["조식메뉴", "중식메뉴", "석식메뉴"]:
    train_merge[menu] = train_merge[menu].apply(reg_modi)
    test_merge[menu] = test_merge[menu].apply(reg_modi)
train_merge.head()

,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,요일_화
0,238,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 된장찌개 쥐어채무침 포기김치,쌀밥/잡곡밥 오징어찌개 쇠불고기 계란찜 청포묵무침 요구르트 포기김치,쌀밥/잡곡밥 육개장 자반고등어구이 두부조림 건파래무침 포기김치,1039.0,331.0,1,0,2476.0,0,0,0,1,0
1,319,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 호박젓국찌개 시래기조림 포기김치,쌀밥/잡곡밥 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이,콩나물밥 양념장 어묵국 유산슬 아삭고추무침 바나나 포기김치,867.0,560.0,1,0,2464.5,0,0,0,0,1
2,111,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 콩나물국 느타리호박볶음 포기김치,카레덮밥 팽이장국 치킨핑거 쫄면야채무침 견과류조림 요구르트 포기김치,쌀밥/잡곡밥 청국장찌개 황태양념구이 고기전 새송이버섯볶음 포기김치,1017.0,573.0,1,0,2455.0,0,0,1,0,0
3,355,모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 근대국 멸치볶음 포기김치,쌀밥/잡곡밥 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치,미니김밥 겨자장 우동 멕시칸샐러드 군고구마 무피클 포기김치,978.0,525.0,1,0,2387.0,0,1,0,0,0
4,34,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 재첩국 방풍나물 포기김치,쌀밥/잡곡밥 떡국 돈육씨앗강정 우엉잡채 청경채무침 요구르트 포기김치,쌀밥/잡곡밥 차돌박이찌개 닭갈비 감자소세지볶음 콩나물무침 포기김치,925.0,330.0,1,0,2232.5,1,0,0,0,0


In [18]:
# for word2vec

def split_str(s):
    s = s.split(" ")
    return s

menus = ["조식메뉴", "중식메뉴", "석식메뉴"]
foods = []
for menu in menus:
    foods += train_merge[menu].apply(split_str).to_list()

In [19]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# load embedding model
#from gensim.models import fasttext
import fasttext
import pickle

def split_str(s):
    s = s.split(" ")
    return s

menus = ["조식메뉴", "중식메뉴", "석식메뉴"]
foods = []
for menu in menus:
    foods += train_merge[menu].apply(split_str).to_list()

filePath = './foods.txt'
foods = [list(set(f)) for f in foods]

model = 0
with open(filePath, 'wb') as lf:
    pickle.dump(foods, lf)

# with open(filePath, 'rb') as lf:
#     readList = pickle.load(lf)
#     model = fasttext.train_unsupervised(readList)
try:
    model.load_model("fast_text_models/foods.bin")
except:
    model = fasttext.train_unsupervised(input = 'foods.txt', model = 'skipgram', dim = 300, epoch = 1000)
    model.save_model("fast_text_models/foods.bin")

In [69]:
# # word2vec
# TRAIN_W2V = True
# model = 0
# try:
#     model = Word2Vec.load('food_embedding.model')
#     print("Model loaded")
# except:
#     if TRAIN_W2V:
#         print("Training w2v")
#         model = Word2Vec(sentences= foods, size = 300, window = 4, min_count = 0, workers = 4)
#         model.save('food_embedding.model')
#     else:
#         print("Model loading failed. Do not train.")

In [70]:
# w2v demo
model["된장찌개"]

array([ 1.58896632e-02, -4.44212072e-02, -3.93766649e-02, -4.08681259e-02,
        4.68341373e-02, -2.42507993e-03,  1.27686951e-02,  5.05425632e-02,
       -4.76542376e-02,  6.60214275e-02,  8.40946212e-02,  3.33735161e-02,
       -6.05189279e-02,  4.89306860e-02, -1.19633600e-03,  4.57714908e-02,
       -3.65049206e-02,  1.86331104e-02, -5.44202095e-03, -3.44981067e-02,
        1.04630115e-02, -2.71369852e-02, -5.88358007e-02, -1.11621611e-01,
        2.33891606e-02,  3.99503075e-02,  3.47270481e-02,  1.24209747e-02,
       -1.34431094e-01,  5.63660376e-02, -3.28778289e-02,  2.43661888e-02,
        1.61481015e-02,  1.94051918e-02, -3.82387266e-02,  1.89314615e-02,
       -6.34373864e-03, -1.41589213e-02, -5.08027384e-04, -3.58604454e-03,
       -5.76537848e-02, -3.36114429e-02,  1.52420849e-02,  4.76286784e-02,
        8.08624271e-03,  2.08855607e-02, -4.04622667e-02,  1.38211725e-02,
       -3.26834037e-03,  2.55024191e-02, -3.79176810e-02,  3.82351391e-02,
        1.60521269e-02,  

In [71]:
when = ["중식", "석식"]
sub_menus = ["밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두유/주스/르트/음료/러드/피클/생채/쌈", 
             "나머지"]

def dish_split_lunch(s):
    s = s.split()
    global sub_menus
    l = {"중식/" + sm : [] for sm in sub_menus}
    for ss in s:
        chk = False
        for cateogory in l:
            for c in cateogory.split("/"):
                if c in ss[-2:]:
                    l[cateogory].append(ss)
                    chk = True
                    break
            if chk:
                break
        if not chk:
            l["중식/나머지"].append(ss)
    
    for key in l:
        l[key] = ' '.join(l[key])
    return l

def dish_split_dinner(s):
    s = s.split()
    global sub_menus
    l = {"석식/" + sm : [] for sm in sub_menus}
    for ss in s:
        chk = False
        for cateogory in l:
            for c in cateogory.split("/"):
                if c in ss[-2:]:
                    l[cateogory].append(ss)
                    chk = True
                    break
            if chk:
                break
        if not chk:
            l["석식/나머지"].append(ss)
    
    for key in l:
        l[key] = ' '.join(l[key])
    return l

train_merge["조식메뉴"] = train_merge["조식메뉴"].apply(dish_split_breakfast)
test_merge["조식메뉴"] = test_merge["조식메뉴"].apply(dish_split_breakfast)

train_merge["중식메뉴"] = train_merge["중식메뉴"].apply(dish_split_lunch)
test_merge["중식메뉴"] = test_merge["중식메뉴"].apply(dish_split_lunch)

train_merge["석식메뉴"] = train_merge["석식메뉴"].apply(dish_split_dinner)
test_merge["석식메뉴"] = test_merge["석식메뉴"].apply(dish_split_dinner)

train_merge

,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,요일_화
0,238,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,1039.0,331.0,1,0,2476.0,0,0,0,1,0
1,319,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,867.0,560.0,1,0,2464.5,0,0,0,0,1
2,111,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,1017.0,573.0,1,0,2455.0,0,0,1,0,0
3,355,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,978.0,525.0,1,0,2387.0,0,1,0,0,0
4,34,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,925.0,330.0,1,0,2232.5,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,4,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,1093.0,421.0,1,0,2418.0,0,0,1,0,0
1202,462,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,832.0,353.0,1,0,2424.5,0,1,0,0,0
1203,1,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,579.0,217.0,1,0,2301.0,1,0,0,0,0
1204,616,{'조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,{'석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두...,1145.0,502.0,1,0,2472.5,0,0,0,1,0


In [72]:
menus = ["조식메뉴", "중식메뉴", "석식메뉴"]
# Convert the column of stringified dicts to dicts
# skip this line, if the column contains dicts
for menu in menus:
    train_merge = pd.concat([train_merge, train_merge[menu].apply(pd.Series)], axis = 1).drop(menu, axis = 1)
    test_merge = pd.concat([test_merge, test_merge[menu].apply(pd.Series)], axis = 1).drop(menu, axis = 1)

In [73]:
def embedding(s):
    return model[s]

when = ["중식", "석식"]
sub_menus = ["밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두유/주스/르트/음료/러드/피클/생채/쌈", 
             "나머지"]
for w in when:
    for sm in sub_menus:
        train_merge[w + "/" + sm] = train_merge[w + "/" + sm].apply(embedding)
        test_merge[w + "/" + sm] = test_merge[w + "/" + sm].apply(embedding)
   
# word2vec
# for menu in menus:
#     for sm in sub_menus:
#         train_merge[menu + "_" + sm] = train_merge[menu].apply(dish_split)
#         test_merge[menu] = test_merge[menu].apply(dish_split)

train_merge

,본사시간외근무명령서승인건수,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,요일_화,조식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두유/주스/르트/음료/러드/피클/생채/쌈,조식/나머지,중식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두유/주스/르트/음료/러드/피클/생채/쌈,중식/나머지,석식/밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두유/주스/르트/음료/러드/피클/생채/쌈,석식/나머지
0,238,1039.0,331.0,1,0,2476.0,0,0,0,1,0,"[0.008252578, -0.043062385, 0.009211329, -0.04...","[0.019070452, 0.0037853352, 0.0014392732, 0.01...","[-0.0026271662, 0.0008327464, 0.0005624727, -0...","[0.0006119643, -0.0043132277, -0.0061980225, -...","[-0.00025498657, -0.00061500724, 0.00023055589...","[0.000825745, -0.0035167388, -0.0050885184, 6...."
1,319,867.0,560.0,1,0,2464.5,0,0,0,0,1,"[-0.0063633085, -0.08558505, 0.0006788304, -0....","[0.030086918, 0.028627932, 0.007940542, -0.008...","[0.00012448526, -8.740279e-05, 0.00036670166, ...","[2.4493182e-05, -0.0001197532, -6.0511225e-05,...","[0.011698902, -0.0033747063, -0.01479184, 0.00...","[0.0005697184, -0.0033954126, -0.004453013, 1...."
2,111,1017.0,573.0,1,0,2455.0,0,0,1,0,0,"[-0.007607466, -0.081600755, -0.0007357609, -0...","[0.019713953, 0.012389395, 0.038397167, -0.001...","[1.1690046e-05, 0.0006352126, -0.00012261949, ...","[0.0007513473, -0.0037321586, -0.004822761, -0...","[-0.00061588595, 6.299102e-05, 0.00046757283, ...","[-0.0002396835, -0.0040301797, -0.0057005105, ..."
3,355,978.0,525.0,1,0,2387.0,0,1,0,0,0,"[-0.017607428, -0.099671826, 0.021119762, -0.0...","[0.020660264, 0.020190071, 0.016116364, -0.001...","[-9.5099684e-05, 0.00017896495, 0.00020475664,...","[0.0056652464, -0.0015695996, 0.00046332926, -...","[-2.5005955e-05, -3.4473753e-05, -3.647806e-05...","[-0.007015029, -0.006265593, -0.007110024, -0...."
4,34,925.0,330.0,1,0,2232.5,1,0,0,0,0,"[-0.009557812, -0.09015174, 0.0007610306, -0.0...","[0.016173586, 0.028458733, 0.00019131618, -0.0...","[0.00017659593, 0.00013278783, 0.0004027581, -...","[0.00079103577, -0.0034707494, -0.004813416, -...","[-0.00020652119, -0.000106880616, 0.0001232694...","[0.0016562231, -0.0038975882, -0.006959452, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,4,1093.0,421.0,1,0,2418.0,0,0,1,0,0,"[-0.0038786228, -0.005201895, -0.0021384764, -...","[0.013581896, 0.0144487, 0.01073638, -0.001719...","[-0.0016962028, -0.0030140674, 0.0029096561, 0...","[5.1342464e-05, -0.00027197105, 6.3935055e-05,...","[-0.009443815, -0.009697585, 0.0022413565, -0....","[0.008677607, 0.0041060615, 0.0036861068, 0.00..."
1202,462,832.0,353.0,1,0,2424.5,0,1,0,0,0,"[-0.0070233704, -0.025129732, 0.02324984, -0.0...","[0.0130629735, 0.00635289, 0.0059369975, 0.009...","[0.00034607574, 0.00043239628, -0.00017536023,...","[0.00034176523, 0.00022855774, 7.452242e-05, -...","[0.0003035021, 0.00023830833, -0.0002055001, -...","[-0.00028157065, -0.002619333, -0.005047947, 0..."
1203,1,579.0,217.0,1,0,2301.0,1,0,0,0,0,"[-0.0026274838, -0.0027899665, 0.00022009193, ...","[0.016096517, 0.01651796, 0.012488103, -0.0021...","[-4.1428142e-05, 4.7118137e-05, 0.00016459987,...","[0.00094294135, -0.0023831378, -0.0036076468, ...","[-0.00028159804, -0.00033226403, 0.00044086372...","[-4.3768273e-06, 4.4051416e-05, -0.00010854615..."
1204,616,1145.0,502.0,1,0,2472.5,0,0,0,1,0,"[-0.0153100435, -0.04286243, 0.013353236, -0.0...","[0.010011371, 0.013576249, 0.010180464, -0.002...","[-0.00042712377, -0.0026149831, 0.0008938481, ...","[1.4564405e-06, 0.00078752293, 0.00084979046, ...","[0.00013105385, -2.0289901e-05, 0.00023113372,...","[0.0005264997, -0.0036867214, -0.004794716, 0...."


In [1]:
# seperate each vecs to new cols
when = ["중식", "석식"]
sub_menus = ["밥/빵/죽/국수/닝롤/게티/이스/우동/찌개/탕/국/스프/계장/개장/우유/두유/주스/르트/음료/러드/피클/생채/쌈", 
             "나머지"]
for w in when:
    for sm in sub_menus:
        tmp_col = w + "/" + sm
        tmp = [tmp_col + str(i) for i in range(300)]
        train_merge[tmp] = pd.DataFrame(train_merge[tmp_col].to_list(), columns = tmp)
        test_merge[tmp] = pd.DataFrame(test_merge[tmp_col].to_list(), columns = tmp)
        train_merge.drop([tmp_col], inplace = True, axis = 1)
        test_merge.drop([tmp_col], inplace = True, axis = 1)

train_merge

NameError: name 'when' is not defined

In [75]:
from sklearn.decomposition import PCA

# PCA 300 -> 20
pca_target = 20
pca = PCA(n_components = pca_target)
for w in when:
    for sm in sub_menus:
        tmp_col = w + "/" + sm
        tmp = [tmp_col + str(i) for i in range(300)]
        target = [tmp_col + "_pca" + str(i) for i in range(pca_target)]
        train_merge[target] = pca.fit_transform(train_merge[tmp])
        test_merge[target] = pca.fit_transform(test_merge[tmp])
        
# del cols before PCA

for w in when:
    for sm in sub_menus:
        tmp_col = w + "/" + sm
        tmp = [tmp_col + str(i) for i in range(300)]
        train_merge.drop(tmp, inplace = True, axis = 1)
        test_merge.drop(tmp, inplace = True, axis = 1)
        
train_merge.head()
        

,본사시간외근무명령서승인건수,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,...,석식/나머지_pca10,석식/나머지_pca11,석식/나머지_pca12,석식/나머지_pca13,석식/나머지_pca14,석식/나머지_pca15,석식/나머지_pca16,석식/나머지_pca17,석식/나머지_pca18,석식/나머지_pca19
0,238,1039.0,331.0,1,0,2476.0,0,0,0,1,...,0.000735,0.000093,-0.000966,-0.000373,0.000090,-0.000577,-0.000212,0.000184,-0.000010,-0.000233
1,319,867.0,560.0,1,0,2464.5,0,0,0,0,...,0.000780,0.000022,-0.001062,0.000245,0.000087,-0.000485,-0.000035,0.000046,-0.000073,-0.000516
2,111,1017.0,573.0,1,0,2455.0,0,0,1,0,...,-0.000907,0.000950,-0.000595,-0.003065,0.000023,-0.000134,-0.000344,-0.000797,0.003833,0.001230
3,355,978.0,525.0,1,0,2387.0,0,1,0,0,...,0.000284,0.001601,-0.007157,0.004531,-0.011801,0.002376,0.003556,0.006338,-0.006712,0.000776
4,34,925.0,330.0,1,0,2232.5,1,0,0,0,...,-0.009539,0.013261,0.008434,-0.002549,-0.006039,0.001933,0.001520,-0.000075,0.002647,0.000897


In [76]:
from sklearn.preprocessing import RobustScaler

target = []

for w in when:
    for sm in sub_menus:
        tmp_col = w + "/" + sm
        target += [tmp_col + "_pca" + str(i) for i in range(pca_target)]
        
scaled = ["본사시간외근무명령서승인건수", 
          #"최저기온(°C)", "평균기온(°C)", "최고기온(°C)", "평균 풍속(m/s)", 
          #"최대 풍속(m/s)", "일강수량(mm)", 
          "본사실근무자수"] + target
train_merge[scaled] = RobustScaler().fit_transform(train_merge[scaled].values)
test_merge[scaled] = RobustScaler().fit_transform(test_merge[scaled].values)

In [77]:
train_merge

,본사시간외근무명령서승인건수,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,...,석식/나머지_pca10,석식/나머지_pca11,석식/나머지_pca12,석식/나머지_pca13,석식/나머지_pca14,석식/나머지_pca15,석식/나머지_pca16,석식/나머지_pca17,석식/나머지_pca18,석식/나머지_pca19
0,-0.135045,1039.0,331.0,1,0,0.011315,0,0,0,1,...,0.234127,0.055959,-0.334606,-0.228741,0.106520,-0.358697,0.249955,0.523255,-0.401000,-0.229807
1,0.045759,867.0,560.0,1,0,-0.053748,0,0,0,0,...,0.269806,-0.076005,-0.520991,0.364511,0.101328,-0.128357,0.667956,0.172284,-0.509282,-0.943003
2,-0.418527,1017.0,573.0,1,0,-0.107496,0,0,1,0,...,-1.089404,1.639081,0.383633,-2.814879,-0.032542,0.748834,-0.063824,-1.965049,6.232873,3.470712
3,0.126116,978.0,525.0,1,0,-0.492221,0,1,0,0,...,-0.129464,2.843224,-12.318341,4.482652,-24.567254,7.031227,9.180797,16.134990,-11.972219,2.322615
4,-0.590402,925.0,330.0,1,0,-1.366337,1,0,0,0,...,-8.044434,24.394432,17.863192,-2.319111,-12.609341,5.921360,4.355392,-0.134654,4.185764,2.628668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,-0.657366,1093.0,421.0,1,0,-0.316832,0,0,1,0,...,6.169528,2.301857,-7.635828,-2.700506,3.282916,-11.840087,0.534676,5.591308,-2.142688,2.539738
1202,0.364955,832.0,353.0,1,0,-0.280057,0,1,0,0,...,-1.366315,-2.650122,0.536666,-0.906938,1.221450,0.779892,0.998159,-1.524158,2.923613,2.891689
1203,-0.664062,579.0,217.0,1,0,-0.978784,1,0,0,0,...,0.780073,-0.718631,-0.551111,0.805460,0.383221,0.046423,0.401057,-0.286201,0.710825,-0.237000
1204,0.708705,1145.0,502.0,1,0,-0.008487,0,0,0,1,...,0.247255,-0.213890,0.077644,0.102173,-0.117127,-0.222305,-1.113513,0.270410,-0.217099,-0.286159


In [206]:

from sklearn.preprocessing import StandardScaler

# std scaling embedded vecs

# scaled_menus_train = {}
# scaled_menus_test = {}

# menus = ["조식메뉴", "중식메뉴", "석식메뉴"]

# for menu in menus:
#     scaled_menus_train[menu] = StandardScaler().fit_transform(train_merge[menu].to_list())
#     scaled_menus_test[menu] = StandardScaler().fit_transform(test_merge[menu].to_list())
#     for idx, val in enumerate(scaled_menus_train[menu]):
#         train_merge[menu][idx] = val
#     for idx, val in enumerate(scaled_menus_test[menu]):
#         test_merge[menu][idx] = val

In [207]:
import pandas_profiling
from pandas_profiling import ProfileReport

# profile = train_merge[["본사시간외근무명령서승인건수", "중식계", "석식계", "최저기온(°C)", "평균기온(°C)", 
#                               "최고기온(°C)", "평균 풍속(m/s)", "최대 풍속(m/s)", "일강수량(mm)", "휴일전후날여부_0.0", "휴일전후날여부_1.0"]].profile_report()
# profile

In [78]:
train_merge.head()

,본사시간외근무명령서승인건수,중식계,석식계,휴일전후날여부_0.0,휴일전후날여부_1.0,본사실근무자수,요일_금,요일_목,요일_수,요일_월,...,석식/나머지_pca10,석식/나머지_pca11,석식/나머지_pca12,석식/나머지_pca13,석식/나머지_pca14,석식/나머지_pca15,석식/나머지_pca16,석식/나머지_pca17,석식/나머지_pca18,석식/나머지_pca19
0,-0.135045,1039.0,331.0,1,0,0.011315,0,0,0,1,...,0.234127,0.055959,-0.334606,-0.228741,0.106520,-0.358697,0.249955,0.523255,-0.401000,-0.229807
1,0.045759,867.0,560.0,1,0,-0.053748,0,0,0,0,...,0.269806,-0.076005,-0.520991,0.364511,0.101328,-0.128357,0.667956,0.172284,-0.509282,-0.943003
2,-0.418527,1017.0,573.0,1,0,-0.107496,0,0,1,0,...,-1.089404,1.639081,0.383633,-2.814879,-0.032542,0.748834,-0.063824,-1.965049,6.232873,3.470712
3,0.126116,978.0,525.0,1,0,-0.492221,0,1,0,0,...,-0.129464,2.843224,-12.318341,4.482652,-24.567254,7.031227,9.180797,16.134990,-11.972219,2.322615
4,-0.590402,925.0,330.0,1,0,-1.366337,1,0,0,0,...,-8.044434,24.394432,17.863192,-2.319111,-12.609341,5.921360,4.355392,-0.134654,4.185764,2.628668


In [ ]:
from pycaret.regression import *

when = ["조식", "중식", "석식"]
sub_menus = ["밥/빵/죽/국수/닝롤/게티/이스/우동", 
             "찌개/탕/국/스프/계장/개장", 
             "우유/두유/주스/음료", 
             "무침/자반/잡채/나물",
             "강정/튀김/가스",
             "조림/볶음/구이/고기", 
             "김치/깍두기/박지/절이", 
             "러드/피클/생채", 
             "나머지"]

exp_lunch = setup(train_merge, target = '중식계',
                 imputation_type='iterative',
                  ignore_features = ["일자"],
                 remove_perfect_collinearity = False,
                 )

In [ ]:
best_4_lunch = compare_models(sort = 'MAE', n_select = 4)

In [ ]:
blended_lunch = blend_models(estimator_list = best_4_lunch, fold = 5)

In [ ]:
prediction_lunch = predict_model(blended_lunch)

In [ ]:
final_lunch = finalize_model(blended_lunch)

In [ ]:
final_prediction_lunch = predict_model(final_lunch, data = test_merge)

In [ ]:
submission["중식계"] = final_prediction_lunch["Label"]

In [ ]:
from pycaret.regression import *
exp_dinner = setup(train_merge, target = '석식계', 
                   ignore_features = ['중식계'])

In [ ]:
best_4_dinner = compare_models(sort = 'MAE', n_select = 4)

In [ ]:
blended_dinner = blend_models(estimator_list = best_4_dinner, fold = 5)

In [ ]:
prediction_dinner = predict_model(blended_dinner)

In [ ]:
final_dinner = finalize_model(blended_dinner)

In [ ]:
final_prediction_dinner = predict_model(final_dinner, data = test_merge)

In [ ]:
submission["석식계"] = final_prediction_dinner["Label"]

In [ ]:
submission.head(20)

In [ ]:
submission.to_csv('submission_성동현.csv', index = False, encoding = "UTF-8")

In [ ]:
test.head(20)

In [ ]:
list(train_merge.columns)

In [90]:
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action = 'ignore')

In [91]:
train_lunch = train_merge['중식계']
train_dinner = train_merge['석식계']

lunch_features = [s for s in train_merge.columns if "중식" in s and "중식계" not in s]
dinner_features = [s for s in train_merge.columns if "석식" in s and "석식계" not in s]

train_lunch_features = train_merge.drop(['중식계', '석식계'] + dinner_features, axis=1)
train_dinner_features = train_merge.drop(['중식계', '석식계'] + lunch_features, axis=1)

test_lunch_features = test_merge.drop(dinner_features, axis=1)
test_dinner_features = test_merge.drop(lunch_features, axis=1)

In [92]:
def print_best_params(model, params, train, train_features):
    grid_model = GridSearchCV(
        model, 
        param_grid = params,
        verbose = 4,
        cv = 5,
        scoring = 'neg_mean_absolute_error')

    grid_model.fit(train_features, train)
    score = -1 * grid_model.best_score_ # mae
    print(
        '{0} 5 CV 시 최적 평균 MAE 값 {1} \n최적 alpha:{2}'.format(model.__class__.__name__, np.round(score, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [93]:
# Simple LGBM Regressor w/o tuning

lgb_params_lunch = {
    'metric' : ['mae'],
    'objective':['mae'],
    'max_depth' : [14 + i for i in range(0, 8, 2)],
    'learning_rate' : [0.016, 0.017],
    'n_estimators':[700 + i for i in range(0, 300, 100)],
    'max_bin' : [550 + i for i in range(0, 250, 50)],
}


In [94]:
lgb_model_lunch = LGBMRegressor()
lgb_estimator_lunch = print_best_params(lgb_model_lunch, lgb_params_lunch, train_lunch, train_lunch_features)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-100.328 total time=   1.8s
[CV 2/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-83.477 total time=   1.7s
[CV 3/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-94.695 total time=   1.7s
[CV 4/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-107.846 total time=   1.7s
[CV 5/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-117.975 total time=   1.6s
[CV 1/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=800, objective=mae;, score=-101.832 total time=   2.0s
[CV 2/5] END learning_rate=0.013, max_bin=550, max_depth=14, metric=mae, n_estimators=800, ob

[CV 4/5] END learning_rate=0.013, max_bin=550, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-110.391 total time=   2.0s
[CV 5/5] END learning_rate=0.013, max_bin=550, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-118.000 total time=   2.0s
[CV 1/5] END learning_rate=0.013, max_bin=600, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-98.886 total time=   1.6s
[CV 2/5] END learning_rate=0.013, max_bin=600, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-83.949 total time=   1.6s
[CV 3/5] END learning_rate=0.013, max_bin=600, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-94.551 total time=   1.7s
[CV 4/5] END learning_rate=0.013, max_bin=600, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-108.471 total time=   1.7s
[CV 5/5] END learning_rate=0.013, max_bin=600, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-118.718 total time=   1.7s
[CV 1/5] END lea

[CV 3/5] END learning_rate=0.013, max_bin=600, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-93.483 total time=   2.5s
[CV 4/5] END learning_rate=0.013, max_bin=600, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-109.242 total time=   2.7s
[CV 5/5] END learning_rate=0.013, max_bin=600, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-119.258 total time=   2.8s
[CV 1/5] END learning_rate=0.013, max_bin=650, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-101.558 total time=   2.1s
[CV 2/5] END learning_rate=0.013, max_bin=650, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-83.389 total time=   2.3s
[CV 3/5] END learning_rate=0.013, max_bin=650, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-94.388 total time=   2.1s
[CV 4/5] END learning_rate=0.013, max_bin=650, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-108.972 total time=   1.8s
[CV 5/5] END lea

[CV 2/5] END learning_rate=0.013, max_bin=650, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-83.423 total time=   2.5s
[CV 3/5] END learning_rate=0.013, max_bin=650, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-93.643 total time=   2.4s
[CV 4/5] END learning_rate=0.013, max_bin=650, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-109.322 total time=   2.4s
[CV 5/5] END learning_rate=0.013, max_bin=650, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-117.743 total time=   2.3s
[CV 1/5] END learning_rate=0.013, max_bin=700, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-100.434 total time=   2.7s
[CV 2/5] END learning_rate=0.013, max_bin=700, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-82.994 total time=   2.0s
[CV 3/5] END learning_rate=0.013, max_bin=700, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-94.022 total time=   1.9s
[CV 4/5] END lear

[CV 1/5] END learning_rate=0.013, max_bin=700, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-101.169 total time=   2.5s
[CV 2/5] END learning_rate=0.013, max_bin=700, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-83.562 total time=   2.4s
[CV 3/5] END learning_rate=0.013, max_bin=700, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-93.869 total time=   2.3s
[CV 4/5] END learning_rate=0.013, max_bin=700, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-107.739 total time=   2.5s
[CV 5/5] END learning_rate=0.013, max_bin=700, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-118.549 total time=   2.3s
[CV 1/5] END learning_rate=0.013, max_bin=750, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-101.450 total time=   2.0s
[CV 2/5] END learning_rate=0.013, max_bin=750, max_depth=14, metric=mae, n_estimators=700, objective=mae;, score=-82.882 total time=   1.8s
[CV 3/5] END lea

...

[CV 4/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-107.216 total time=   2.2s
[CV 5/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-117.219 total time=   2.2s
[CV 1/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-102.705 total time=   2.6s
[CV 2/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-82.787 total time=   2.5s
[CV 3/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-93.051 total time=   2.7s
[CV 4/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-108.147 total time=   3.0s
[CV 5/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-117.026 total time=   2.9s
[CV 1/5] END le

[CV 3/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-94.285 total time=   2.5s
[CV 4/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-108.597 total time=   2.5s
[CV 5/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-117.145 total time=   2.6s
[CV 1/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-101.762 total time=   2.5s
[CV 2/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-82.736 total time=   2.3s
[CV 3/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-94.474 total time=   2.5s
[CV 4/5] END learning_rate=0.017, max_bin=600, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-108.569 total time=   2.6s
[CV 5/5] END lea

[CV 2/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-83.777 total time=   2.0s
[CV 3/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-94.318 total time=   1.9s
[CV 4/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-109.591 total time=   2.1s
[CV 5/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-117.895 total time=   2.0s
[CV 1/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-101.413 total time=   2.4s
[CV 2/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-82.895 total time=   2.9s
[CV 3/5] END learning_rate=0.017, max_bin=650, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-94.577 total time=   2.6s
[CV 4/5] END lear

[CV 1/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-98.846 total time=   1.8s
[CV 2/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-83.022 total time=   1.7s
[CV 3/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-93.462 total time=   1.6s
[CV 4/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-108.361 total time=   1.6s
[CV 5/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-117.768 total time=   1.7s
[CV 1/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-99.383 total time=   1.9s
[CV 2/5] END learning_rate=0.017, max_bin=700, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-82.485 total time=   1.8s
[CV 3/5] END learn

[CV 5/5] END learning_rate=0.017, max_bin=750, max_depth=18, metric=mae, n_estimators=900, objective=mae;, score=-117.752 total time=   2.8s
[CV 1/5] END learning_rate=0.017, max_bin=750, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-99.355 total time=   2.4s
[CV 2/5] END learning_rate=0.017, max_bin=750, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-83.040 total time=   2.5s
[CV 3/5] END learning_rate=0.017, max_bin=750, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-92.566 total time=   2.3s
[CV 4/5] END learning_rate=0.017, max_bin=750, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-108.377 total time=   2.4s
[CV 5/5] END learning_rate=0.017, max_bin=750, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-118.094 total time=   2.2s
[CV 1/5] END learning_rate=0.017, max_bin=750, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-100.080 total time=   2.8s
[CV 2/5] END lea

In [95]:
lgb_preds_lunch = lgb_estimator_lunch.predict(test_lunch_features, axis=1)

In [96]:
submission["중식계"] = lgb_preds_lunch

In [97]:
# Simple LGBM Regressor w/o tuning

lgb_params_dinner = {
    'metric' : ['mae'],
    'objective':['mae'],
    'max_depth' : [16 + i for i in range(0, 8, 2)],
    'learning_rate' : [0.013, 0.015, 0.017],
    'n_estimators':[700 + i for i in range(0, 400, 100)],
    'max_bin' : [500 + i for i in range(0, 200, 50)],
}

In [98]:
lgb_model_dinner = LGBMRegressor()
lgb_estimator_dinner = print_best_params(lgb_model_dinner, lgb_params_dinner, train_dinner, train_dinner_features)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV 1/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-75.158 total time=   2.2s
[CV 2/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-55.213 total time=   2.2s
[CV 3/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-55.384 total time=   2.2s
[CV 4/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-60.892 total time=   2.2s
[CV 5/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-105.329 total time=   2.3s
[CV 1/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=800, objective=mae;, score=-75.312 total time=   2.5s
[CV 2/5] END learning_rate=0.013, max_bin=500, max_depth=16, metric=mae, n_estimators=800, objec

[CV 4/5] END learning_rate=0.013, max_bin=500, max_depth=20, metric=mae, n_estimators=1000, objective=mae;, score=-60.765 total time=   3.3s
[CV 5/5] END learning_rate=0.013, max_bin=500, max_depth=20, metric=mae, n_estimators=1000, objective=mae;, score=-102.436 total time=   3.4s
[CV 1/5] END learning_rate=0.013, max_bin=500, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-73.480 total time=   2.4s
[CV 2/5] END learning_rate=0.013, max_bin=500, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-56.893 total time=   2.4s
[CV 3/5] END learning_rate=0.013, max_bin=500, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-55.467 total time=   2.4s
[CV 4/5] END learning_rate=0.013, max_bin=500, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-60.474 total time=   2.4s
[CV 5/5] END learning_rate=0.013, max_bin=500, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-104.530 total time=   2.4s
[CV 1/5] END lea

[CV 3/5] END learning_rate=0.013, max_bin=550, max_depth=18, metric=mae, n_estimators=1000, objective=mae;, score=-55.695 total time=   3.5s
[CV 4/5] END learning_rate=0.013, max_bin=550, max_depth=18, metric=mae, n_estimators=1000, objective=mae;, score=-61.809 total time=   3.1s
[CV 5/5] END learning_rate=0.013, max_bin=550, max_depth=18, metric=mae, n_estimators=1000, objective=mae;, score=-102.498 total time=   3.1s
[CV 1/5] END learning_rate=0.013, max_bin=550, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-72.378 total time=   2.2s
[CV 2/5] END learning_rate=0.013, max_bin=550, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-55.348 total time=   2.3s
[CV 3/5] END learning_rate=0.013, max_bin=550, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-55.739 total time=   2.3s
[CV 4/5] END learning_rate=0.013, max_bin=550, max_depth=20, metric=mae, n_estimators=700, objective=mae;, score=-60.831 total time=   2.3s
[CV 5/5] END lea

[CV 2/5] END learning_rate=0.013, max_bin=600, max_depth=16, metric=mae, n_estimators=1000, objective=mae;, score=-55.356 total time=   3.7s
[CV 3/5] END learning_rate=0.013, max_bin=600, max_depth=16, metric=mae, n_estimators=1000, objective=mae;, score=-55.997 total time=   3.7s
[CV 4/5] END learning_rate=0.013, max_bin=600, max_depth=16, metric=mae, n_estimators=1000, objective=mae;, score=-61.495 total time=   2.9s
[CV 5/5] END learning_rate=0.013, max_bin=600, max_depth=16, metric=mae, n_estimators=1000, objective=mae;, score=-103.531 total time=   3.0s
[CV 1/5] END learning_rate=0.013, max_bin=600, max_depth=18, metric=mae, n_estimators=700, objective=mae;, score=-74.599 total time=   2.0s
[CV 2/5] END learning_rate=0.013, max_bin=600, max_depth=18, metric=mae, n_estimators=700, objective=mae;, score=-55.496 total time=   2.0s
[CV 3/5] END learning_rate=0.013, max_bin=600, max_depth=18, metric=mae, n_estimators=700, objective=mae;, score=-55.522 total time=   2.0s
[CV 4/5] END le

[CV 1/5] END learning_rate=0.013, max_bin=600, max_depth=22, metric=mae, n_estimators=1000, objective=mae;, score=-74.125 total time=   3.5s
[CV 2/5] END learning_rate=0.013, max_bin=600, max_depth=22, metric=mae, n_estimators=1000, objective=mae;, score=-55.762 total time=   3.5s
[CV 3/5] END learning_rate=0.013, max_bin=600, max_depth=22, metric=mae, n_estimators=1000, objective=mae;, score=-55.504 total time=   3.4s
[CV 4/5] END learning_rate=0.013, max_bin=600, max_depth=22, metric=mae, n_estimators=1000, objective=mae;, score=-61.229 total time=   3.7s
[CV 5/5] END learning_rate=0.013, max_bin=600, max_depth=22, metric=mae, n_estimators=1000, objective=mae;, score=-105.153 total time=   3.3s
[CV 1/5] END learning_rate=0.013, max_bin=650, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-74.981 total time=   2.1s
[CV 2/5] END learning_rate=0.013, max_bin=650, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-55.580 total time=   2.6s
[CV 3/5] END l

...

[CV 3/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-55.219 total time=   3.1s
[CV 4/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-62.231 total time=   2.8s
[CV 5/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-106.159 total time=   2.7s
[CV 1/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=900, objective=mae;, score=-75.977 total time=   3.0s
[CV 2/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=900, objective=mae;, score=-55.772 total time=   3.3s
[CV 3/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=900, objective=mae;, score=-55.035 total time=   3.1s
[CV 4/5] END learning_rate=0.017, max_bin=500, max_depth=22, metric=mae, n_estimators=900, objective=mae;, score=-62.215 total time=   3.3s
[CV 5/5] END learni

[CV 2/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-55.927 total time=   2.0s
[CV 3/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-54.988 total time=   2.0s
[CV 4/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-61.427 total time=   2.0s
[CV 5/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=800, objective=mae;, score=-106.386 total time=   2.1s
[CV 1/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-74.841 total time=   2.4s
[CV 2/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-56.057 total time=   2.5s
[CV 3/5] END learning_rate=0.017, max_bin=550, max_depth=20, metric=mae, n_estimators=900, objective=mae;, score=-54.899 total time=   2.4s
[CV 4/5] END learni

[CV 1/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=800, objective=mae;, score=-73.923 total time=   2.0s
[CV 2/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=800, objective=mae;, score=-54.989 total time=   2.0s
[CV 3/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=800, objective=mae;, score=-55.300 total time=   2.0s
[CV 4/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=800, objective=mae;, score=-62.548 total time=   2.0s
[CV 5/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=800, objective=mae;, score=-104.018 total time=   2.0s
[CV 1/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=900, objective=mae;, score=-74.190 total time=   2.2s
[CV 2/5] END learning_rate=0.017, max_bin=600, max_depth=18, metric=mae, n_estimators=900, objective=mae;, score=-55.128 total time=   2.2s
[CV 3/5] END learni

[CV 5/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=700, objective=mae;, score=-106.400 total time=   1.2s
[CV 1/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=800, objective=mae;, score=-74.415 total time=   1.4s
[CV 2/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=800, objective=mae;, score=-55.638 total time=   1.3s
[CV 3/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=800, objective=mae;, score=-55.103 total time=   1.4s
[CV 4/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=800, objective=mae;, score=-61.550 total time=   1.5s
[CV 5/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=800, objective=mae;, score=-105.064 total time=   1.5s
[CV 1/5] END learning_rate=0.017, max_bin=650, max_depth=16, metric=mae, n_estimators=900, objective=mae;, score=-74.314 total time=   2.0s
[CV 2/5] END learn

[CV 4/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-61.482 total time=   1.7s
[CV 5/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=700, objective=mae;, score=-106.856 total time=   1.8s
[CV 1/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-74.367 total time=   1.9s
[CV 2/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-55.432 total time=   2.0s
[CV 3/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-55.062 total time=   1.9s
[CV 4/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-61.231 total time=   2.0s
[CV 5/5] END learning_rate=0.017, max_bin=650, max_depth=22, metric=mae, n_estimators=800, objective=mae;, score=-105.309 total time=   1.9s
[CV 1/5] END learn

In [99]:
lgb_preds_dinner = lgb_estimator_dinner.predict(test_dinner_features, axis=1)

In [100]:
submission["석식계"] = lgb_preds_dinner
submission

,일자,중식계,석식계
0,2021-01-27,854.049589,370.451868
1,2021-01-28,805.773523,471.866215
2,2021-01-29,687.301884,323.594838
3,2021-02-01,1176.827979,529.495346
4,2021-02-02,931.955804,531.749281
5,2021-02-03,843.977652,390.011995
6,2021-02-04,825.267679,485.709840
7,2021-02-05,735.644669,362.973400
8,2021-02-08,1182.793925,541.334301
9,2021-02-09,905.473356,539.711472


In [101]:
submission.to_csv('submission_성동현.csv', index = False, encoding = "UTF-8")